## 缺失值问题
- 描述
- 判断
- 处理

## 读取并观察数据

In [1]:
import numpy as np
import pandas as pd
import sklearn

In [5]:
data_orig = pd.read_csv(r"./data.csv",encoding = 'gb18030'
                       #,index_col = 0
                       )   #读取文件
data_orig.head()    #观察一下前5行

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [6]:
data_orig['status'][:5]  #观察一下结果标签

0    1
1    0
2    1
3    0
4    1
Name: status, dtype: int64

In [7]:
data_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 90 columns):
Unnamed: 0                                    4754 non-null int64
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility

### 数据集特征：
- index没有从0开始，也不连续。reindex一下
- 规模4754*90. 样本量4754, 原始特征90
- 数值型特征70(float64) + 13(int64) , 非数值型 7个
- 缺失值：比较明显的就一个student_feature，直接dropna
- 其余特征大部分在若干样本上isnull,若干的范围在300以内，所以含有nan的行全部删除试试

## 缺失值处理

In [8]:
data_orig.loc[:10,'student_feature']

0     NaN
1     1.0
2     NaN
3     NaN
4     NaN
5     1.0
6     NaN
7     NaN
8     NaN
9     1.0
10    NaN
Name: student_feature, dtype: float64

In [12]:
data = data_orig.dropna(axis=1,thresh=4000) #去掉了student_feature这一特征后的DF为data

In [15]:
data.dropna(axis=0,inplace=True)  #只要有NA的行就去掉
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3983 entries, 0 to 4753
Data columns (total 89 columns):
Unnamed: 0                                    3983 non-null int64
custid                                        3983 non-null int64
trade_no                                      3983 non-null object
bank_card_no                                  3983 non-null object
low_volume_percent                            3983 non-null float64
middle_volume_percent                         3983 non-null float64
take_amount_in_later_12_month_highest         3983 non-null int64
trans_amount_increase_rate_lately             3983 non-null float64
trans_activity_month                          3983 non-null float64
trans_activity_day                            3983 non-null float64
transd_mcc                                    3983 non-null float64
trans_days_interval_filter                    3983 non-null float64
trans_days_interval                           3983 non-null float64
regional_mobility

D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
data.describe()

,Unnamed: 0,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
count,3983.000000,3.983000e+03,3983.000000,3983.000000,3983.000000,3983.000000,3983.000000,3983.000000,3983.000000,3983.000000,...,3983.000000,3983.000000,3983.00000,3983.000000,3983.000000,3983.000000,3983.000000,3983.000000,3983.000000,3983.000000
mean,5978.966106,1.680845e+06,0.019982,0.904783,2019.093648,4.378511,0.806465,0.388897,18.207381,26.118755,...,3481.621893,1842.186292,9325.05649,76.685413,4.953050,5.473763,16614.838062,8082.725082,23.431835,52.370073
std,3437.230356,1.032941e+06,0.026783,0.132394,4047.801495,78.108940,0.194997,0.163907,4.117520,17.958326,...,1419.273829,537.812515,7282.72002,12.806453,2.963372,3.401296,14480.490287,5601.708872,37.416001,51.011847
min,5.000000,1.140000e+02,0.000000,0.050000,0.000000,0.000000,0.230000,0.051000,5.000000,5.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.000000,-2.000000
25%,3082.000000,7.510880e+05,0.010000,0.880000,0.000000,0.610000,0.670000,0.263000,16.000000,15.000000,...,2500.000000,1566.000000,5000.00000,77.000000,3.000000,3.000000,7800.000000,4800.000000,5.000000,9.000000
50%,5970.000000,1.620145e+06,0.010000,0.960000,600.000000,0.960000,0.860000,0.375000,18.000000,22.000000,...,3300.000000,1820.000000,7800.00000,79.000000,5.000000,5.000000,14400.000000,7100.000000,14.000000,33.000000
75%,8963.500000,2.560038e+06,0.020000,0.990000,2000.000000,1.580000,1.000000,0.497000,20.000000,31.000000,...,4400.000000,2100.000000,11800.00000,80.000000,7.000000,8.000000,20400.000000,9937.500000,23.000000,87.000000
max,11992.000000,4.004694e+06,0.340000,1.000000,68000.000000,4677.980000,1.000000,0.941000,42.000000,229.000000,...,10000.000000,6900.000000,87100.00000,87.000000,18.000000,20.000000,266400.000000,82800.000000,360.000000,323.000000


### 缺失值初步处理完
#### 接下来应该先把数值型无量纲化呢还是先把非数值型数值化？
#### 非数值型一共有7个：trade_no; bank_card_no;reg_preference_for_trad; source;id_name;latest_query_time;loans_credit_limit

## 去除唯一化（看书发现的）
比如trade_no每个用户都不一样，说明对最后的预测结果没有影响

In [20]:
data.drop(columns = 'trade_no',inplace=True)
data.head()

,Unnamed: 0,custid,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,卡号1,0.01,0.99,0,0.90,0.55,0.313,17.0,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,卡号1,0.02,0.94,2000,1.28,1.00,0.458,19.0,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,卡号1,0.04,0.96,0,1.00,1.00,0.114,13.0,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,卡号1,0.00,0.96,2000,0.13,0.57,0.777,22.0,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,卡号1,0.01,0.99,0,0.46,1.00,0.175,13.0,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


## 文字型转数值型
- 先分别看看那7个object的特征（哦，不，现在还剩6个）

In [23]:
data.loc[:9,'bank_card_no']

0    卡号1
1    卡号1
2    卡号1
3    卡号1
4    卡号1
5    卡号1
8    卡号1
9    卡号1
Name: bank_card_no, dtype: object

In [64]:
print(set(data['bank_card_no']))

{'卡号1'}


### 后来发现：bank_card_no所有用户也都一样

In [65]:
data.drop(columns='bank_card_no',inplace=True)

In [24]:
data.loc[:5,'reg_preference_for_trad']

0    一线城市
1    一线城市
2    一线城市
3    三线城市
4    一线城市
5    三线城市
Name: reg_preference_for_trad, dtype: object

In [34]:
data.loc[:9,'source']

0    xs
1    xs
2    xs
3    xs
4    xs
5    xs
8    xs
9    xs
Name: source, dtype: object

### 发现'source'特征似乎所有用户都一样
如果是这样的话，对结果类别没什么影响，应该删掉，经过判断确实一样。

In [62]:
tmp = set(data['source'])

KeyError: 'source'

In [33]:
print(tmp)

{'xs'}


In [35]:
data=data.drop(columns = 'source')

In [66]:
data.shape

(3983, 85)

In [36]:
data.loc[:5,'id_name']

0     蒋红
1    崔向朝
2    王中云
3    何洋洋
4     赵洋
5     刘成
Name: id_name, dtype: object

In [37]:
# 同上，去掉
data.drop(columns='id_name',inplace=True)

In [67]:
data.shape #验证删除成功

(3983, 85)

In [40]:
data.loc[:5,'latest_query_time'] #

0    2018-04-25
1    2018-05-03
2    2018-05-05
3    2018-05-05
4    2018-04-15
5    2018-05-04
Name: latest_query_time, dtype: object

In [42]:
data.loc[:5,'loans_latest_time']

0    2018-04-19
1    2018-05-05
2    2018-05-01
3    2018-05-03
4    2018-01-07
5    2018-05-03
Name: loans_latest_time, dtype: object

### 将'latest_query_time'、'loans_latest_time'两个日期处理成时间戳 
将时间字符串转化成时间戳分为两步： 
第一步：将时间字符串转换成时间元组 strptime  
第二步：将时间元组转换成时间戳类型 mktime

In [44]:
import time
data['loans_latest_timestamp'] = data['loans_latest_time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d')))

In [45]:
data.shape

(3983, 87)

In [46]:
data.loc[:5,['loans_latest_time','loans_latest_timestamp']]

,loans_latest_time,loans_latest_timestamp
0,2018-04-19,1.524067e+09
1,2018-05-05,1.525450e+09
2,2018-05-01,1.525104e+09
3,2018-05-03,1.525277e+09
4,2018-01-07,1.515254e+09
5,2018-05-03,1.525277e+09


In [54]:
#用data['loans_latest_timestamp']替换data['loans_latest_time']
data['loans_latest_time']=data['loans_latest_timestamp']

In [58]:
data.drop(columns='loans_latest_time',inplace=True)
data.shape

(3983, 86)

In [59]:
#同法处理data['latest_query_time'] 干脆直接修改data['latest_query_time']
data['latest_query_time'] = data['latest_query_time'].apply(lambda x:time.mktime(time.strptime(x,'%Y-%m-%d')))

In [68]:
data.shape

(3983, 85)

### 7个object只剩下了reg_preference_for_trad


In [70]:
print(set(data['reg_preference_for_trad']))

{'二线城市', '境外', '一线城市', '三线城市', '其他城市'}


In [71]:
#对data['reg_preference_for_trad']采用labelEncoder
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['reg_preference_for_trad']=le.fit_transform(data['reg_preference_for_trad'])

In [73]:
data.loc[:5,'reg_preference_for_trad']

0    0
1    0
2    0
3    1
4    0
5    1
Name: reg_preference_for_trad, dtype: int32

In [74]:
le.classes_

array(['一线城市', '三线城市', '二线城市', '其他城市', '境外'], dtype=object)

### 非数值型特征处理小结：
'trade_no'/'id_name'对每个用户基本都是unique的;'bank_card_no'/'source'每个用户都一样。所以它们四个对分类无意义，直接drop掉
'latest_query_time'/'loans_latest_time'统一编程了timestamp格式，是flaot64
'reg_preference_for_trad'里面有五类，利用LabelEncoder变成了0/1/2/3/4

## 下考虑归一化（有选择的）
但是是不是应该在特征选择之后呢？此时应该看一看群里的讨论